In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import split, regexp_replace, col, when, sum

In [3]:
spark = SparkSession.builder.appName("ReadHDFSFile").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 14:20:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# competitions = spark.read.csv("hdfs://localhost:9000/big-data/competitions.csv",header=True)
# events = spark.read.csv("hdfs://localhost:9000/big-data/events.csv",header=True)
frames = spark.read.csv("hdfs://localhost:9000/big-data/frames.csv",header=True)
# lineups = spark.read.csv("hdfs://localhost:9000/big-data/lineups.csv",header=True)
# matches = spark.read.csv("hdfs://localhost:9000/big-data/matches.csv",header=True)

In [5]:
frames.show()

+--------------------+--------------------+--------+--------+-----+------+--------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|            location|
+--------------------+--------------------+--------+--------+-----+------+--------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False|  True|[22.6975416639254...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[46.5466540256494...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[47.4452205797475...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[48.8195457108986...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[49.7459543709158...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[53.4689340941401...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[59.3929593689272...|
|25dfc952-7a85-464...|[82.0588035227948.

### Splitting Location

In [6]:
fs = frames.first()
fs['location']

'[22.697541663925428, 41.01277368603656]'

In [7]:
frames = frames.withColumn("location_clean", regexp_replace(col("location"), r"[\[\]]", "")) \
               .withColumn("x", split(col("location_clean"), ", ").getItem(0).cast("double")) \
               .withColumn("y", split(col("location_clean"), ", ").getItem(1).cast("double")) \
               .drop("location_clean").drop("location")

frames.show(22)

+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|                 x|                 y|
+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False|  True|22.697541663925428| 41.01277368603656|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 46.54665402564941| 46.65992069066397|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 47.44522057974756| 64.52929870627902|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 48.81954571089864|35.810976155696935|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 49.74595437091587|27.859841158806844|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|53.468934094140124|48.

### Replacing True/False with 1/0

In [8]:
frames.printSchema()

root
 |-- id: string (nullable = true)
 |-- visible_area: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- teammate: string (nullable = true)
 |-- actor: string (nullable = true)
 |-- keeper: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)



In [11]:
columns_re = ['teammate','actor','keeper']

for column in columns_re:
    frames = frames.withColumn(column,when(col(column)=='True',1) \
    .when(col(column) == 'False',0).otherwise(col(column)))

frames.show(5)

+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|                 x|                 y|
+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     1|22.697541663925428| 41.01277368603656|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 46.54665402564941| 46.65992069066397|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 47.44522057974756| 64.52929870627902|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 48.81954571089864|35.810976155696935|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 49.74595437091587|27.859841158806844|
+--------------------+--------------------+--------+--------+-----+------+------------------+---

In [60]:
# According to documentation, visible_area data is in format
# X1, Y1, X2, Y2 ... Xn, Yn

In [ ]:
null_counts = frames.select(
    *[
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in frames.columns
    ]
)
null_counts.show()

+---+------------+--------+--------+-----+------+---+---+
| id|visible_area|match_id|teammate|actor|keeper|  x|  y|
+---+------------+--------+--------+-----+------+---+---+
|  0|           0|       0|       0|    0|     0|  0|  0|
+---+------------+--------+--------+-----+------+---+---+



In [6]:
spark.stop()